In [26]:
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import PDBList
import numpy as np
from scipy.spatial.transform import Rotation

In [14]:
pdbid = '2q4g'
chain = 'Y'
repeats = "2-30,31-58,59-87,88-115,116-144,145-172,173-201,202-229,230-258,259-286,287-315,316-343,344-372,373-400,401-429,430-455"
repeats = repeats.split(',')
repeats = [[int(idx.split('-')[0]), int(idx.split('-')[1])] for idx in repeats]
path = '/home/bio'
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdbid, pdir=path, file_format='pdb')
structure = PDBParser(QUIET=True).get_structure(pdbid, path + "/pdb/{}.pdb".format(pdbid))

Desired structure doesn't exist


In [3]:
selected_chain = structure[0][chain]
selected_residues = [res for res in selected_chain if res.id[0] == " "] # Exclude hetero groups and solvent
#selected_atoms = [atom for residue in selected_residues for atom in residue.get_atoms()]

In [20]:
sup = SVDSuperimposer()


In [4]:
CA_pos = []
for res in selected_residues:
    CA_pos.append(list(res['CA'].get_vector()))
CA_pos = np.array(CA_pos)
CA_pos

array([[40.02899933, 66.65399933, -7.58500004],
       [40.98899841, 66.28099823, -3.92400002],
       [41.83599854, 68.98300171, -1.38300002],
       ...,
       [70.87000275, 64.84100342, 18.00399971],
       [67.98200226, 64.83899689, 20.47200012],
       [68.51300049, 66.33499908, 23.92399979]])

Basic proof of concept for getting the translation matricies that super-imposes one unit onto the next.\
TODO: Probably explore using vinilla SVD from scipy to get all of the principal vectors so the torsion angles can be calculated according to this paper;
https://onlinelibrary.wiley.com/doi/10.1002/jcc.20237 \
also SVD could be unstable if two principal directions have similar magnitude leading to axes being permutated. this would need to be investigated if this is actually a problem, or if another method of getting unit orientation needs to be worked out.
more info on calculating torsion angles;
https://www.youtube.com/watch?v=NSU0OnW9yLk


In [33]:
for i in range(len(repeats) - 1):
    print(f"start: {repeats[i][0]} end: {repeats[i][1]}")
    x = CA_pos[repeats[1][0]:repeats[1][1]+1,:] # compare to constant refrence
    #x = CA_pos[repeats[i][0]:repeats[i][1]+1,:] # compare consecuitive units
    y = CA_pos[repeats[i+1][0]:repeats[i+1][1]+1,:]
    min_len = min(x.shape[0], y.shape[0])
    sup.set(x[0:min_len,:],y[0:min_len,:])
    sup.run()
    rot, tran = sup.get_rotran()
    print(Rotation.from_matrix(rot).as_euler('xyz', degrees=True))
    print(sup.get_rms())

start: 2 end: 30
[2.12895929e-15 0.00000000e+00 3.35576870e-15]
5.567585431281865e-15
start: 31 end: 58
[10.60353611  7.1784923  14.20340711]
1.9946231013279503
start: 59 end: 87
[12.54213776  9.12186992 35.71250996]
0.8048565196023998
start: 88 end: 115
[26.3629371  12.13177285 49.8124651 ]
1.775551708233427
start: 116 end: 144
[23.17833634  7.12621479 68.25783158]
0.9347210121042736
start: 145 end: 172
[36.18823676  5.20982894 86.19644706]
1.5234843371875706
start: 173 end: 201
[ 30.46230124  -4.83860572 103.81079032]
0.9081724066283013
start: 202 end: 229
[ 40.36132362  -7.55923786 118.10703203]
1.713198926058325
start: 230 end: 258
[ 32.73224262 -14.31634412 138.42227007]
1.0494587132352233
start: 259 end: 286
[ 41.32873758 -18.1379075  159.10189417]
1.960044811875689
start: 287 end: 315
[ 29.81894344 -23.47110775 177.8759162 ]
1.2253490330171815
start: 316 end: 343
[  33.55572411  -22.64647885 -164.46154847]
1.9768536140431958
start: 344 end: 372
[  25.55169608  -30.20603766 -148.